In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime

# ---------------------- 配色方案（上海：粉色系渐变，缺失数据淡灰色） ----------------------
# 价格区间从低到高：浅粉→深粉；缺失数据：淡灰色
colors = ['#ffddee', '#ffbbcc', '#ff99bb', '#ff77aa', '#ff5599', '#f0e6e6']  # 淡粉→深粉→淡灰

# ---------------------- 1. 数据读取与预处理 ----------------------
filtered_files = [f for f in os.listdir(os.getcwd()) if f.startswith("pet_funeral_shanghai_cleaned_20251113_140525")]
if not filtered_files:
    print("❌ No Shanghai filtered files found!")
    exit()
latest_file = max(filtered_files, key=lambda x: os.path.getmtime(x))
df = pd.read_excel(latest_file)
total = len(df)
print(f"✅ Loaded Shanghai data: {latest_file} (Total: {total} merchants in Shanghai)")

# 提取有效价格
def extract_valid_price(price_val):
    if pd.isna(price_val) or str(price_val).strip() in ['人均 -', '', 'None']:
        return None
    match = re.search(r'(\d+)', str(price_val))
    return float(match.group()) if match else None

df['valid_price'] = df['avg_price'].apply(extract_valid_price)
valid_data = df[df['valid_price'].notna()]['valid_price']
missing_count = total - len(valid_data)

# ---------------------- 2. 数据分组 ----------------------
price_bins = [0, 500, 800, 1200, 1500, float('inf')]
price_labels = ['0-500 RMB', '501-800 RMB', '801-1200 RMB', '1201-1500 RMB', '>1500 RMB', 'Missing Data']
price_groups = pd.cut(valid_data, bins=price_bins, labels=price_labels[:-1], right=False)
price_counts = price_groups.value_counts().sort_index()
price_counts['Missing Data'] = missing_count

# ---------------------- 3. 绘制柱状图 ----------------------
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial', 'Helvetica']
plt.rcParams['axes.unicode_minus'] = False

fig, ax = plt.subplots(figsize=(12, 7))
bars = ax.bar(
    price_counts.index, 
    price_counts.values, 
    color=colors,  # 浅粉→深粉→淡灰
    width=0.65,
    edgecolor='white', 
    linewidth=1.5, 
    alpha=0.9
)

# 数据标签（数量+百分比）
for bar in bars:
    height = bar.get_height()
    percentage = (height / total) * 100
    ax.text(
        bar.get_x() + bar.get_width()/2, 
        height + 0.5,
        f"{int(height)}\n({percentage:.1f}%)",
        ha='center', 
        va='bottom', 
        fontsize=9, 
        fontweight='bold', 
        color='#5d4037'  # 棕褐色文字，与粉色对比清晰
    )

# 标题与轴标签
ax.set_title(
    'Per Capita Price Distribution of Pet Funeral Merchants in Shanghai',
    fontsize=16, 
    fontweight='bold', 
    color='#ff5599'  # 最深粉色，强化识别
)
ax.set_xlabel('Price Range', fontsize=12, fontweight='medium')
ax.set_ylabel('Number of Merchants', fontsize=12, fontweight='medium')

# 美化细节
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', labelsize=10, rotation=30)
ax.tick_params(axis='y', labelsize=10)
ax.set_ylim(0, max(price_counts.values) * 1.2)

# ---------------------- 4. 保存图片 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(os.getcwd(), f'shanghai_price_distribution_{timestamp}.png')
plt.tight_layout()
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()
print(f"📁 Shanghai price chart saved to: {save_path}")

✅ Loaded Shanghai data: pet_funeral_shanghai_cleaned_20251113_140525.xlsx (Total: 124 merchants in Shanghai)
📁 Shanghai price chart saved to: /Users/syx/Documents/python脚本/shanghai_price_distribution_20251114_021504.png


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime

# ---------------------- 配色方案（与价格图统一：浅粉→深粉，缺失数据淡粉灰） ----------------------
colors = [
    '#ffeff2',  # 最浅粉（<3.5）
    '#ffd9e0',  # 浅粉（3.5-4.0）
    '#ffb8c7',  # 中粉（4.0-4.5）
    '#ff6d93',  # 最深粉（4.5-5.0，最高评分）
    '#f0e6e6'   # 淡粉灰（缺失数据）
]

# ---------------------- 1. 数据读取与预处理 ----------------------
filtered_files = [f for f in os.listdir(os.getcwd()) if f.startswith("pet_funeral_shanghai_cleaned_20251113_140525")]
if not filtered_files:
    print("❌ No Shanghai filtered files found!")
    exit()
latest_file = max(filtered_files, key=lambda x: os.path.getmtime(x))
df = pd.read_excel(latest_file)
total = len(df)
print(f"✅ Loaded Shanghai data: {latest_file} (Total: {total} merchants in Shanghai)")

# 提取有效评分
def extract_valid_rating(star_val):
    if pd.isna(star_val) or str(star_val).strip() in ['0.0', '0.0分', '无评分', '', 'None']:
        return None
    match = re.search(r'(\d+\.?\d*)', str(star_val))
    return float(match.group()) if match and float(match.group()) > 0 else None

df['valid_rating'] = df['star'].apply(extract_valid_rating)
valid_data = df[df['valid_rating'].notna()]['valid_rating']
missing_count = total - len(valid_data)

# ---------------------- 2. 数据分组 ----------------------
rating_bins = [0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ['<3.5(Poor)', '3.5-4.0(Fair)', '4.0-4.5(Good)', '4.5-5.0 (Excellent)', 'Missing Data']
rating_groups = pd.cut(valid_data, bins=rating_bins, labels=rating_labels[:-1], right=False)
rating_counts = rating_groups.value_counts().sort_index()
rating_counts['Missing Data'] = missing_count

# ---------------------- 3. 绘制饼图 ----------------------
plt.style.use('seaborn-v0_8-white')
plt.rcParams['font.family'] = ['Arial', 'Helvetica']

fig, ax = plt.subplots(figsize=(9, 9))
wedgeprops = {'edgecolor': 'white', 'linewidth': 2}  # 白色边框区分扇区

wedges, texts, autotexts = ax.pie(
    rating_counts.values,
    labels=rating_labels,
    colors=colors,  # 浅粉→深粉→淡粉灰
    autopct='%1.1f%%',
    startangle=90,
    pctdistance=0.82,
    labeldistance=1.1,
    wedgeprops=wedgeprops,
    textprops={'fontsize': 11}
)

# 美化标签
for autotext in autotexts:
    autotext.set_color('white')  # 白色百分比文字，与粉色系对比清晰
    autotext.set_fontweight('bold')
for text in texts:
    text.set_fontweight('bold')
    text.set_color('#6d4c41')  # 深棕文字，与淡粉灰协调

# 标题与图例
ax.set_title(
    'Distribution of Pet Funeral Merchant Ratings in Shanghai',
    fontsize=16, 
    fontweight='bold', 
    color='#ff6d93'  # 最深粉色，与最高评分区间呼应
)
legend_labels = [f'{label} (n={int(rating_counts[i])})' for i, label in enumerate(rating_labels)]
ax.legend(legend_labels, loc='center left', bbox_to_anchor=(1, 0.5),
          fontsize=10, frameon=True, edgecolor='#ffd9e0')  # 图例边框用浅粉，强化风格

# ---------------------- 4. 保存图片 ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(os.getcwd(), f'shanghai_rating_distribution_{timestamp}.png')
plt.tight_layout()
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()
print(f"📁 Shanghai rating chart saved to: {save_path}")

✅ Loaded Shanghai data: pet_funeral_shanghai_cleaned_20251113_140525.xlsx (Total: 124 merchants in Shanghai)


/var/folders/9q/ywtdr2kx78d4gg56pbn1dk_w0000gn/T/ipykernel_35643/342614032.py:78: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  legend_labels = [f'{label} (n={int(rating_counts[i])})' for i, label in enumerate(rating_labels)]


📁 Shanghai rating chart saved to: /Users/syx/Documents/python脚本/shanghai_rating_distribution_20251114_021809.png


In [14]:
import pandas as pd
import os
from pyecharts import options as opts
from pyecharts.charts import Map
from datetime import datetime

# ---------------------- 配置路径（替换为上海清洗后的数据路径） ----------------------
INPUT_CLEANED_DATA = "/Users/syx/Documents/python脚本/shanghai_second_cleaned_20251113.xlsx"  # 上海清洗后的数据路径
OUTPUT_HEATMAP = f"上海宠物店铺热力图_{datetime.now().strftime('%Y%m%d')}.html"


# ---------------------- 上海所有行政区列表（确保无遗漏） ----------------------
all_shanghai_districts = [
    "黄浦区", "徐汇区", "长宁区", "静安区", "普陀区", "虹口区", "杨浦区",
    "闵行区", "宝山区", "嘉定区", "浦东新区", "金山区", "松江区", "青浦区",
    "奉贤区", "崇明区"
]


# ---------------------- 生成上海热力图（粉色系渐变，与北京蓝系区分） ----------------------
if os.path.exists(INPUT_CLEANED_DATA):
    df = pd.read_excel(INPUT_CLEANED_DATA)
    if "district_cn" in df.columns:
        # 统计各行政区店铺数量（无数据的区默认为0）
        district_counts = df["district_cn"].value_counts().to_dict()
        for dist in all_shanghai_districts:
            if dist not in district_counts:
                district_counts[dist] = 0
        heatmap_data = [(k, v) for k, v in district_counts.items()]

        # 粉色系渐变配色（与北京蓝系呼应，风格统一）
        pink_gradient = [
            "#fff0f5", "#ffd1dc", "#ffb6c1", "#ff9bb3", 
            "#ff7da6", "#ff6098", "#e64a8a", "#cc337c"
        ]

        c = (
            Map(
                init_opts=opts.InitOpts(
                    width="1200px",
                    height="700px",
                    page_title="Shanghai Pet Funeral Merchants Regional Distribution Heatmap"
                )
            )
            .add(
                series_name="Number of Merchants",  # 图例英文
                data_pair=heatmap_data,
                maptype="上海",  # 地图类型改为上海
                label_opts=opts.LabelOpts(
                    is_show=True, 
                    font_size=10,
                    color="#333"
                )
            )
            .set_global_opts(
                title_opts=opts.TitleOpts(
                    title="Distribution of Pet Funeral Stores in Shanghai",  # 标题英文
                    pos_left="center",
                    pos_top="15px",
                    title_textstyle_opts=opts.TextStyleOpts(
                        font_size=18,
                        color="#cc337c",  # 标题色与最深粉呼应
                        font_weight="bold"
                    )
                ),
                legend_opts=opts.LegendOpts(
                    pos_bottom="10px",
                    pos_left="center",
                    textstyle_opts=opts.TextStyleOpts(color="#ff7da6")  # 图例文字色适配粉色
                ),
                visualmap_opts=opts.VisualMapOpts(
                    min_=0,
                    max_=max([v for _, v in heatmap_data]) if heatmap_data else 1,
                    range_color=pink_gradient,  # 应用粉色渐变
                    pos_right="20px",
                    pos_top="center",
                    orient="vertical",
                    is_calculable=True,
                    textstyle_opts=opts.TextStyleOpts(color="#ff7da6")
                ),
                toolbox_opts=opts.ToolboxOpts(
                    is_show=True,
                    feature={
                        "zoom": {"show": True, "title": "放大"},
                        "restore": {"show": True, "title": "重置"}
                    },
                    pos_right="30px",
                    pos_top="50px",
                    item_size=15
                )
            )
            .set_series_opts(
                # 悬停提示框：区域名（中文）在上，英文数量在下
                tooltip_opts=opts.TooltipOpts(
                    formatter=lambda params: f"""
                    <div style="text-align:center; min-width:100px;">
                        <div style="font-weight:bold; color:#cc337c;">{params.name}</div>
                        <div style="color:#ff7da6;">{
                            "No Merchants" if params.value == 0 
                            else f"Number of Merchants：{params.value}"
                        }</div>
                    </div>
                    """,
                    background_color="rgba(255, 235, 245, 0.9)",  # 浅粉半透明背景
                    border_color="#ffb6c1",  # 边框色与中间粉匹配
                    textstyle_opts=opts.TextStyleOpts(color="#cc337c")
                ),
                itemstyle_opts=opts.ItemStyleOpts(
                    border_color="#fff",
                    border_width=1
                )
            )
        )

        c.render(OUTPUT_HEATMAP)
        print(f"✅ 上海热力图已保存：{OUTPUT_HEATMAP}")
    else:
        print("❌ 数据中无 'district_cn' 列，请检查上海清洗步骤")
else:
    print(f"❌ 未找到文件：{INPUT_CLEANED_DATA}")

✅ 上海热力图已保存：上海宠物店铺热力图_20251114.html


In [1]:
import pandas as pd
import os
from pyecharts import options as opts
from pyecharts.charts import Map
from datetime import datetime

# ---------------------- 配置路径 ----------------------
INPUT_CLEANED_DATA = "/Users/syx/Documents/python脚本/shanghai_second_cleaned_20251113.xlsx"  # 上海清洗文件路径
OUTPUT_HEATMAP = f"上海宠物店铺热力图_{datetime.now().strftime('%Y%m%d')}.html"


# ---------------------- 上海所有行政区列表 ----------------------
all_shanghai_districts = [
    "黄浦区", "徐汇区", "长宁区", "静安区", "普陀区", "虹口区", "杨浦区",
    "闵行区", "宝山区", "嘉定区", "浦东新区", "金山区", "松江区", "青浦区",
    "奉贤区", "崇明区"
]


# ---------------------- 生成上海粉色系热力图（与北京/广州风格统一） ----------------------
if os.path.exists(INPUT_CLEANED_DATA):
    df = pd.read_excel(INPUT_CLEANED_DATA)
    if "district_cn" in df.columns:
        # 统计各行政区店铺数量（无数据的区默认为0）
        district_counts = df["district_cn"].value_counts().to_dict()
        for dist in all_shanghai_districts:
            if dist not in district_counts:
                district_counts[dist] = 0
        heatmap_data = [(k, v) for k, v in district_counts.items()]

        # 粉色渐变配色（8色，与北京蓝/广州绿保持渐变节奏一致）
        pink_gradient = [
            "#fff5f7", "#fff0f5", "#ffe6f2", "#ffccd5",
            "#ffb3c8", "#ff8fab", "#ff6b9b", "#cc337c"
        ]

        c = (
            Map(
                init_opts=opts.InitOpts(
                    width="1200px",
                    height="700px",
                    page_title="Shanghai Pet Funeral Merchants Regional Distribution Heatmap"
                )
            )
            .add(
                series_name="Number of Merchants",
                data_pair=heatmap_data,
                maptype="上海",
                label_opts=opts.LabelOpts(is_show=True, font_size=10, color="#333"),
                # 标记点颜色与粉色渐变统一（核心：区域颜色=标记点颜色）
                itemstyle_opts=opts.ItemStyleOpts(
                    color=pink_gradient,  # 绑定粉色渐变列表
                    border_color="#fff",
                    border_width=1
                )
            )
            .set_global_opts(
                title_opts=opts.TitleOpts(
                    title="Distribution of Pet Funeral Stores in Shanghai",
                    pos_left="center",
                    pos_top="15px",
                    title_textstyle_opts=opts.TextStyleOpts(
                        font_size=18,
                        color="#cc337c",  # 标题色=最深粉（对应北京最深蓝/广州最深绿）
                        font_weight="bold"
                    )
                ),
                legend_opts=opts.LegendOpts(
                    pos_bottom="10px",
                    pos_left="center",
                    textstyle_opts=opts.TextStyleOpts(
                        color="#ff8fab"  # 图例文字色=中粉（对应北京中蓝/广州中绿）
                    )
                ),
                visualmap_opts=opts.VisualMapOpts(
                    min_=0,
                    max_=max([v for _, v in heatmap_data]) if heatmap_data else 1,
                    range_color=pink_gradient,  # 视觉映射=粉色渐变
                    pos_right="20px",
                    pos_top="center",
                    orient="vertical",
                    is_calculable=True,
                    textstyle_opts=opts.TextStyleOpts(color="#ff8fab")
                ),
                toolbox_opts=opts.ToolboxOpts(
                    is_show=True,
                    feature={
                        "zoom": {"show": True, "title": "放大"},
                        "restore": {"show": True, "title": "重置"}
                    },
                    pos_right="30px",
                    pos_top="50px",
                    item_size=15
                )
            )
            .set_series_opts(
                # 悬停提示框：粉色系风格（与北京/广州格式完全一致）
                tooltip_opts=opts.TooltipOpts(
                    formatter=lambda params: f"""
                    <div style="text-align:center; min-width:100px;">
                        <div style="font-weight:bold; color:#cc337c;">{params.name}</div>
                        <div style="color:#ff8fab;">{
                            "暂无店铺" if params.value == 0 
                            else f"Number of Merchants：{params.value}"
                        }</div>
                    </div>
                    """,
                    background_color="rgba(255, 240, 245, 0.9)",  # 浅粉半透明背景
                    border_color="#ffccd5",  # 边框色=中浅粉
                    textstyle_opts=opts.TextStyleOpts(color="#cc337c")
                )
            )
        )

        c.render(OUTPUT_HEATMAP)
        print(f"✅ 上海热力图已保存：{OUTPUT_HEATMAP}")
    else:
        print("❌ 数据中无 'district_cn' 列，请检查清洗步骤")
else:
    print(f"❌ 未找到文件：{INPUT_CLEANED_DATA}")

✅ 上海热力图已保存：上海宠物店铺热力图_20251114.html
